In [1]:
import json
import pandas as pd
from csv import writer
import os


In [13]:
current_periods = []
current_teams = tuple()
nullToken = None


In [17]:
def valueOrNull(key, obj):
        if key in obj:
            value = obj[key]
        else:
            value = nullToken
        return value


In [ ]:
def append_header(file_name):
        with open(file_name, 'a+', newline='') as write_obj:
            csv_writer = writer(write_obj)
            csv_writer.writerow(["gameId", "season", "teamHome", "teamAway", "eventType", "eventTeam", "period", "periodTime", "eventSide", "coordinateX", "coordinateY", "shooterName", "goalieName", "shotType", "emptyNet", "strength"])


In [18]:
def getPlayInfo(play):
        about = play["about"]
        periodTime = about["periodTime"]
        period = about["period"]
        eventTeam = play["team"]["name"]
        if(period == 5):
            eventSide = "shootout"
        else:
            team = current_teams.index(eventTeam)
            current_period = current_periods[period - 1]
            if team == 0:
                eventSide = valueOrNull("rinkSide", current_period["home"])
            else:
                eventSide = valueOrNull("rinkSide", current_period["away"])
        result = play["result"]
        eventType = result["event"]
        if eventType == "Goal":
            eventType = 1
        else:
            eventType = 0
        coordinates = play["coordinates"]
        coordinateX = valueOrNull("x", coordinates)
        coordinateY = valueOrNull("y", coordinates)
        shooterName = play["players"][0]["player"]["fullName"]
        goalieName = play["players"][-1]["player"]["fullName"]
        shotType = valueOrNull("secondaryType", result)
        if eventType == 1:
            emptyNet = valueOrNull("emptyNet", result)
            strength = result["strength"]["code"]
        else:
            emptyNet = nullToken
            strength = nullToken
        return [eventType, eventTeam, period, periodTime, eventSide, coordinateX, coordinateY, shooterName, goalieName, shotType, emptyNet, strength]


In [37]:
import numpy as np
def get_distance(x1,x2, y1,y2):
    """
    Calculates the euclidean distance from the event coordinates (x,y) to the goal, 
    based on what side of the rink (Eventside) the event is.
    Returns the euclidean distance rounded to 4 decimal places.
    """
    coord1 = np.array([x1,x2])
    coord2 = np.array([y1,y2])
    
    
    # Distance is the norm of a-b
    distance = np.linalg.norm(coord1 - coord2)
    
    return round(distance,4)

In [40]:
#Get time differece
import datetime as dt
def get_time_diff(t1, t2):
    start_dt = dt.datetime.strptime(t1, '%M:%S')
    end_dt = dt.datetime.strptime(t2, '%M:%S')
    diff = (end_dt - start_dt) 
    return diff.seconds 

In [44]:
for filename in os.listdir("raw_data"):
    # Parse the filename
    name = filename.split('.json')[0]
    is_json = filename[-5:] == '.json'
    print(filename)
    # If it's a json:
    if is_json:
        # Open and load file
        json_file = open("raw_data/"+filename)
        data = json.load(json_file)
        with open("test.csv", 'w+', newline='') as write_obj:
                    csv_writer = writer(write_obj)
                    gameId = data["gamePk"]
                    season = data["gameData"]["game"]["season"]
                    teamHome = data["gameData"]["teams"]["home"]["name"]
                    teamAway = data["gameData"]["teams"]["away"]["name"]
                    current_teams = (teamHome, teamAway)
                    periods = data["liveData"]["linescore"]["periods"]
                    current_periods = periods
                    allplays=data["liveData"]["plays"]["allPlays"]
                    for i in range(0,len(allplays)):
                        if allplays[i]["result"]["event"] == "Shot" or allplays[i]["result"]["event"] == "Goal":
                            play= getPlayInfo(allplays[i])
                            last_coordinates = allplays[i-1]["coordinates"]
                            last_coordinateX = valueOrNull("x", last_coordinates)
                            last_coordinateY = valueOrNull("y", last_coordinates)
                            last_event = allplays[i-1]["result"]["event"]
                            last_periodTime = allplays[i-1]["about"]["periodTime"]
                            print("----",last_coordinates, last_periodTime, last_event)
                            print(play)
                            if last_coordinateX is None :
                                last_coordinateX = play[5]
                            if last_coordinateY is None :
                                last_coordinateY = play[6]
                            distance = get_distance(play[5], play[6], last_coordinateX, last_coordinateY)
                            time_last_event = get_time_diff(last_periodTime, play[3])
                            rebound=False
                            if last_event=="Shot":
                                rebound=True
                            speed=distance/time_last_event
                            csv_writer.writerow([gameId,season,teamHome,teamAway] + play + [last_coordinateX,
                                                                                           last_coordinateY,
                                                                                           last_event,last_periodTime,
                                                                                           distance,time_last_event,
                                                                                           rebound, speed])
    json_file.close()
    break              
                            


2015020001.json


PermissionError: [Errno 13] Permission denied: 'test.csv'